# Alternating K Groups

This notebook contains the experiment to investigate the question **What is the benefit of restricting alternating random groups?**, which is reported on Figure 6 in the [paper](https://arxiv.org/abs/2004.07641).

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [ ]:
import numpy as np
import pickle
import multiprocessing

In [ ]:
from lib.mobilitysim import MobilitySimulator
from lib.parallel import launch_parallel_simulations
from lib.distributions import CovidDistributions
from lib.data import collect_data_from_df
from lib.measures import (MeasureList, Interval,
                          SocialDistancingForKGroups,
                          SocialDistancingForPositiveMeasure)
from lib.inference import gen_initial_seeds

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
# Set it
np.random.seed(c)
# Define prefix string used to save plots
runstr = f'run{c}_'  

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 1 # Set to at least 20 to obtain stable results

#### Define the number of parallel workers used for simulations 

In [ ]:
num_workers = multiprocessing.cpu_count() - 1

#### Set the time to simulate

In [ ]:
days_present = 35  # Days since March 10 on April 12 (March 10 is time 0 for our experiments)
days_future = 12 * 7  # Projecting 12 weeks into the future
max_time_future = 24.0 * days_future  # in hours

#### Define population/mobility parameters

Import downsampled version of Tübingen (population downsampled by 20x, sites by 10x) that was used used to generate mobility traces on the fly for inference (sites, home locations, etc)

In [ ]:
case_downsample = 10                                
mob_settings = 'lib/mobility/Tubingen_settings_10_10_hh.pk'  
area = 'TU'
country = 'GER'

# See town-generator.ipynb for an example on how to create the settings
with open(mob_settings, 'rb') as fp:
    obj = pickle.load(fp)
mob = MobilitySimulator(**obj)

#### Load the case data

We adjust the number of cases to the down-scaling of the population

In [ ]:
new_cases_ = collect_data_from_df(country=country, area=area, datatype='new',
    start_date_string='2020-03-10', until=days_present)
new_cases = np.ceil(1/case_downsample * new_cases_)

# Instantiate correct state transition distributions (estimated from in literature)
distributions = CovidDistributions(country=country)

#### Define initial seed count per state 

These numbers are based on the *downscaled* infection counts on April 12.

In [ ]:
with open('summaries/'+f'ave_present_seeds_{c}.pk', 'rb') as logfile:
    present_seeds = pickle.load(logfile)

#### Set epidemic parameters

Inferred using Bayesian optimization.

In [ ]:
beta = 1.1383
inferred_params = {
    'betas' : {'education': beta,'social': beta,'bus_stop': beta,'office': beta,'supermarket': beta}, # site infectivity by type
    'mu': 1.0
}

#### Define standard testing parameters

The parameters are the same used for inference.

In [ ]:
def standard_testing(max_time):
    standard_testing_params = {
        'testing_t_window'    : [0.0, max_time], # in hours
        'testing_frequency'   : 24.0,     # in hours
        'test_reporting_lag'  : 48.0,     # in hours (actual and self-report delay)
        'tests_per_batch'     : 10,       # assume 300 tests/day in LK Tübingen
        'test_smart_delta'    : 24.0 * 3, # in hours
        'test_smart_duration' : 24.0 * 7, # in hours
        'test_smart_action'   : 'isolate', 
        'test_smart_num_contacts'   : 10, 
        'test_targets'        : 'isym',
        'test_queue_policy'   : 'fifo',
        'smart_tracing'       : None, 
    }
    return standard_testing_params

#### Define helper I/O functions to load/save the output of the experiment

In [ ]:
def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

---

## 2. Run the simulations

Define a helper function to run general type of experiment, fixing all parameters above.


In [ ]:
def run(tparam, measure_list, t, local_seeds, dynamic_tracing=False):

    # add standard measure of positives staying isolated
    measure_list +=  [
        SocialDistancingForPositiveMeasure(
            t_window=Interval(0.0, t), p_stay_home=1.0)
    ]
    measure_list = MeasureList(measure_list)

    # run simulations
    summary = launch_parallel_simulations(
        mob_settings, 
        distributions, 
        random_repeats, num_workers, 
        inferred_params, local_seeds, tparam, measure_list, 
        max_time=t, 
        num_people=mob.num_people, 
        num_sites=mob.num_sites, 
        site_loc=mob.site_loc, 
        home_loc=mob.home_loc,
        dynamic_tracing=dynamic_tracing,
        verbose=False)
    return summary
    

Use settings as above and simulate in the future, with additional measures of the same length

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
# baseline (no measure for the future starting April 12)
future_baseline = run(standard_testing(max_time_future), [], max_time_future, present_seeds) 

In [ ]:
K_groups = [2, 3, 4]
testing_params_SD_4 = standard_testing(max_time_future)
summaries_SD_4 = []
K_groups_weeks = 6

# no quarantine baseline (standard measures)
summaries_SD_4.append(future_baseline)

# alternating quarantines
for j, k in enumerate(K_groups):
    
    # same as tuned plus containing K groups alternatinglye in 4 week period afterwards
    m = [SocialDistancingForKGroups(
         t_window=Interval(0.0, (24.0 * 7 * K_groups_weeks)), 
         K=k)] 

    res = run(testing_params_SD_4, m, max_time_future, present_seeds) 
    summaries_SD_4.append(res)
    print(k, ' done.')
    
save_summary(summaries_SD_4, 'summaries_SD_4.pk')

---

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Reproduce Figure 6 in the paper.

In [ ]:
summaries_SD_4 = load_summary('summaries_SD_4.pk')
plotter = Plotter()
K_groups = [2, 3, 4]
titles_SD_4 = ['No curfew'] + [f'K = {k}' for k in K_groups]
plotter.compare_total_infections(
    summaries_SD_4, 
    titles=titles_SD_4, 
    figtitle=r'Curfew of K random groups for staggered K - 1 day windows',
    filename=runstr + 'SD_050', 
    figsize=(6,4), acc=500, 
    ymax=500, errorevery=14)